In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%d\t%s" % (word_sum, current_key)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%d\t%s" % (word_sum, current_key)

Writing reducer.py


In [4]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
OUT_DIR2="wordcount_result_"$(date +"%s%6N")

NUM_REDUCERS2=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordRating" \
    -D mapreduce.job.reduces=${NUM_REDUCERS2} \
    -mapper "sort" \
    -reducer "sort -rn" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null

hdfs dfs -cat ${OUT_DIR2}/part-00000 |head -7|tail -1|awk '{ printf ("%s\t%d\n", $2, $1) }'

is	126420


rm: `wordcount_result_1512716525581991': No such file or directory
17/12/08 07:02:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/08 07:02:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/08 07:02:10 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/08 07:02:10 INFO mapreduce.JobSubmitter: number of splits:2
17/12/08 07:02:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512669625839_0011
17/12/08 07:02:11 INFO impl.YarnClientImpl: Submitted application application_1512669625839_0011
17/12/08 07:02:11 INFO mapreduce.Job: The url to track the job: http://436c512f30cf:8088/proxy/application_1512669625839_0011/
17/12/08 07:02:11 INFO mapreduce.Job: Running job: job_1512669625839_0011
17/12/08 07:02:17 INFO mapreduce.Job: Job job_1512669625839_0011 running in uber mode : false
17/12/08 07:02:17 INFO mapreduce.Job:  map 0% reduce 0%
17/12/08 07:02:33 INFO mapreduce.Job:  map 30% reduce 0%
17/12/08 07:0